In [1]:
import pandas as pd
import yaml
from model import YNet
from datetime import datetime
from utils.preprocessing import load_raw_dataset
import time

tic = time.time()

FOLDERNAME = './'
time_stamp = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
CHECKPOINT = None # FOLDERNAME + 'pretrained_models/2022_01_27_23_58_00_weights.pt' # None means no checkpoint will be used to fine-tune
CONFIG_FILE_PATH = 'config/sdd_raw_train.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = time_stamp  # arbitrary name for this experiment
DATASET_NAME = 'sdd'
SDD_RAW_PATH = FOLDERNAME + "data/sdd_raw"
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
print(f"Experiment {EXPERIMENT_NAME} has started")

if params['use_raw_data']:
    TRAIN_IMAGE_PATH = FOLDERNAME + 'data/sdd_raw/annotations'
    TEST_IMAGE_PATH = FOLDERNAME + 'data/sdd_raw/annotations'
else:
    TEST_DATA_PATH = FOLDERNAME + 'data/SDD/test_trajnet.pkl'
    TEST_IMAGE_PATH = FOLDERNAME + 'data/SDD/test'  # only needed for YNet, PECNet ignores this value
params['segmentation_model_fp'] = FOLDERNAME + 'ynet_additional_files/segmentation_models/SDD_segmentation.pth'
OBS_LEN = 8  # in timesteps
PRED_LEN = 12  # in timesteps
NUM_GOALS = 20  # K_e
NUM_TRAJ = 1  # K_a
ROUNDS = 1  # Y-net is stochastic. How often to evaluate the whole dataset
BATCH_SIZE = 8




Experiment 2022_01_29_02_20_06 has started


In [2]:

if params['use_raw_data']:
    train_data, val_data = load_raw_dataset(path=SDD_RAW_PATH, step=params['step'],
                                  window_size=params['min_num_steps_seq'], stride=params['filter_stride'],
                                  train_labels=params['train_labels'], test_labels=params['test_labels'],
                                  test_per=params['test_per'], max_train_agents=params['max_train_agents'],
                                  train_set_ratio=params['train_set_ratio'], test_on_train=params['test_on_train'],
                                  num_train_agents=params['num_train_agents'], num_test_agents=params['num_test_agents'],
                                  random_train_test=params['random_train_test_split'])
else:
	train_data = pd.read_pickle(TRAIN_IMAGE_PATH)
	val_data = pd.read_pickle(TEST_DATA_PATH)

2662 agents for each training class, 518 agents for test class


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)
if CHECKPOINT: model.load(CHECKPOINT)
model.train_style_enc(train_data, val_data, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=TEST_IMAGE_PATH,
				experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
				device='cpu', dataset_name=DATASET_NAME, use_raw_data=params['use_raw_data'])

toc = time.time()
print(time.strftime("%Hh%Mm%Ss", time.gmtime(toc - tic)))

/home/jonas/miniconda3/envs/rebut/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jonas/miniconda3/envs/rebut/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/jonas/miniconda3/envs/rebut/lib/python3.8/site-packages/torch/serialization.py:671: SourceChangeWarning: source code of class 'torch.nn.modules.batchnorm.BatchNorm2d' has change

Preprocess data


Prepare Dataset: 100%|██████████| 424/424 [00:00<00:00, 1469.45it/s]
/home/jonas/vita_epfl_causal/ynet/utils/dataloader.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]/home/jonas/miniconda3/envs/rebut/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training


RuntimeError: torch.cat(): Sizes of tensors must match except in dimension 1. Got 32 and 31 in dimension 2 (The offending index is 1)